# Airfoil's Aerodynamic Coefficient Prediction using ANNs

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
# importing the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
# reading the csv file
df = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA5D_05/NACA5D_05.csv")

In [4]:
# printing the first 10 rows of the dataset
df.head(10)

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yL1,yL2,yL3,yL4,yL5,alpha,M,Re,CL,CD,Cm
0,5,0.0,5,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-10.0,0.1,100000,-1.140,0.02494,0.005
1,5,0.0,5,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-9.0,0.1,100000,-1.027,0.02116,0.004
2,5,0.0,5,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-8.0,0.1,100000,-0.914,0.01697,0.004
3,5,0.0,5,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-7.0,0.1,100000,-0.800,0.01364,0.003
4,5,0.0,5,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-6.0,0.1,100000,-0.687,0.02483,0.003
5,5,0.0,5,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-5.0,0.1,100000,-0.572,0.01565,0.002
6,5,0.0,5,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-4.0,0.1,100000,-0.458,0.01114,0.002
7,5,0.0,5,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-3.0,0.1,100000,-0.344,0.00882,0.001
8,5,0.0,5,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-2.0,0.1,100000,-0.229,0.01391,0.001
9,5,0.0,5,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-1.0,0.1,100000,-0.115,0.01357,0.000


In [5]:
# checking for missing values in the dataset
df.isnull().sum()

t            0
CL_design    0
p            0
s            0
yU1          0
yU2          0
yU3          0
yU4          0
yU5          0
yL1          0
yL2          0
yL3          0
yL4          0
yL5          0
alpha        0
M            0
Re           0
CL           0
CD           0
Cm           0
dtype: int64

In [6]:
# description of the dataset
df.describe()

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yL1,yL2,yL3,yL4,yL5,alpha,M,Re,CL,CD,Cm
count,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.000000,236880.00000,236880.000000,236880.000000,236880.000000,236880.000000
mean,19.960106,0.841223,27.632979,0.493351,0.096292,0.134144,0.111651,0.060456,0.017849,-0.034706,-0.063388,-0.066130,-0.045857,-0.016689,0.000000,0.20000,300000.000000,0.219178,0.025908,0.008664
std,10.003265,0.595420,16.860962,0.499957,0.047779,0.066417,0.059813,0.035302,0.011050,0.042542,0.065845,0.057175,0.033677,0.010987,6.055313,0.08165,141421.654747,0.925885,0.011283,0.092522
min,5.000000,0.000000,5.000000,0.000000,-0.004196,-0.034220,-0.048766,-0.048585,-0.021718,-0.150412,-0.221016,-0.208757,-0.158776,-0.062862,-10.000000,0.10000,100000.000000,-2.607000,0.002690,-0.169000
25%,10.000000,0.200000,15.000000,0.000000,0.060440,0.083429,0.066202,0.035288,0.009724,-0.064663,-0.113478,-0.110225,-0.069669,-0.024271,-5.000000,0.10000,200000.000000,-0.485000,0.018250,-0.031000
50%,20.000000,0.800000,25.000000,0.000000,0.094692,0.132999,0.110292,0.059885,0.017656,-0.030806,-0.064636,-0.066175,-0.045224,-0.016381,-0.000000,0.20000,300000.000000,0.218000,0.024650,-0.004000
75%,30.000000,1.400000,45.000000,1.000000,0.127479,0.179109,0.154377,0.086256,0.026085,-0.006198,-0.019232,-0.023071,-0.020494,-0.008568,5.000000,0.30000,400000.000000,0.928000,0.031580,0.018000
max,35.000000,1.800000,75.000000,1.000000,0.237894,0.331768,0.289322,0.160810,0.046274,0.077967,0.135837,0.101022,0.036191,0.008290,10.000000,0.30000,500000.000000,3.020000,0.617020,0.453000


In [7]:
# shuffling all the rows
df = df.sample(frac=1)
df.head(10)

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yL1,yL2,yL3,yL4,yL5,alpha,M,Re,CL,CD,Cm
96339,10,0.6,35,0,0.047380,0.080088,0.072908,0.041155,0.012554,-0.020241,-0.018954,-0.015398,-0.011583,-0.004600,2.0,0.3,300000,0.571,0.01533,-0.056
13240,5,0.0,35,0,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-0.0,0.1,100000,0.000,0.01336,-0.000
49664,20,0.2,35,0,0.072448,0.109192,0.097923,0.057722,0.018512,-0.061304,-0.088714,-0.078543,-0.047595,-0.015752,10.0,0.1,500000,1.399,0.02784,-0.032
195756,15,1.6,5,1,0.132697,0.104186,0.031919,-0.015464,-0.010499,0.053716,-0.050304,-0.103421,-0.095133,-0.037371,5.0,0.1,200000,-0.183,0.01525,0.359
219800,10,1.8,15,0,0.124699,0.149966,0.111743,0.060361,0.017723,0.043334,0.050287,0.022558,0.007080,0.000402,4.0,0.3,200000,1.371,0.02474,-0.062
63090,25,0.4,5,1,0.098113,0.129986,0.102672,0.052489,0.015349,-0.058570,-0.117012,-0.119685,-0.080409,-0.027930,-4.0,0.2,500000,-0.774,0.00994,0.109
134839,30,1.0,15,1,0.161214,0.207909,0.170221,0.093411,0.028148,-0.021457,-0.088130,-0.098929,-0.067641,-0.024148,9.0,0.1,100000,1.709,0.04955,0.028
235221,10,1.8,45,1,0.037522,0.059956,0.056694,0.035262,0.012574,-0.029343,-0.039015,-0.031538,-0.017402,-0.004564,-10.0,0.3,200000,-1.109,0.02621,-0.022
183534,30,1.4,25,1,0.190848,0.272148,0.222440,0.105637,0.026151,-0.018009,-0.021588,-0.054515,-0.058237,-0.026198,5.0,0.3,500000,1.534,0.04382,0.034
7646,20,0.0,15,1,0.066841,0.099001,0.088267,0.052665,0.017139,-0.066714,-0.098883,-0.088200,-0.052642,-0.017135,-8.0,0.1,500000,-1.028,0.02365,0.012


In [8]:
# dropping columns t, CL_design, p & s
df = df.drop(columns=['t', 'CL_design', 'p', 's'], axis=1)
df.head(10)

,yU1,yU2,yU3,yU4,yU5,yL1,yL2,yL3,yL4,yL5,alpha,M,Re,CL,CD,Cm
96339,0.047380,0.080088,0.072908,0.041155,0.012554,-0.020241,-0.018954,-0.015398,-0.011583,-0.004600,2.0,0.3,300000,0.571,0.01533,-0.056
13240,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-0.0,0.1,100000,0.000,0.01336,-0.000
49664,0.072448,0.109192,0.097923,0.057722,0.018512,-0.061304,-0.088714,-0.078543,-0.047595,-0.015752,10.0,0.1,500000,1.399,0.02784,-0.032
195756,0.132697,0.104186,0.031919,-0.015464,-0.010499,0.053716,-0.050304,-0.103421,-0.095133,-0.037371,5.0,0.1,200000,-0.183,0.01525,0.359
219800,0.124699,0.149966,0.111743,0.060361,0.017723,0.043334,0.050287,0.022558,0.007080,0.000402,4.0,0.3,200000,1.371,0.02474,-0.062
63090,0.098113,0.129986,0.102672,0.052489,0.015349,-0.058570,-0.117012,-0.119685,-0.080409,-0.027930,-4.0,0.2,500000,-0.774,0.00994,0.109
134839,0.161214,0.207909,0.170221,0.093411,0.028148,-0.021457,-0.088130,-0.098929,-0.067641,-0.024148,9.0,0.1,100000,1.709,0.04955,0.028
235221,0.037522,0.059956,0.056694,0.035262,0.012574,-0.029343,-0.039015,-0.031538,-0.017402,-0.004564,-10.0,0.3,200000,-1.109,0.02621,-0.022
183534,0.190848,0.272148,0.222440,0.105637,0.026151,-0.018009,-0.021588,-0.054515,-0.058237,-0.026198,5.0,0.3,500000,1.534,0.04382,0.034
7646,0.066841,0.099001,0.088267,0.052665,0.017139,-0.066714,-0.098883,-0.088200,-0.052642,-0.017135,-8.0,0.1,500000,-1.028,0.02365,0.012


In [9]:
# defining the feature and target columns
X = df.drop(columns=['CL', 'CD', 'Cm'])
y = df[['CL', 'CD', 'Cm']]

In [10]:
# displaying the features table
X.head(10)

,yU1,yU2,yU3,yU4,yU5,yL1,yL2,yL3,yL4,yL5,alpha,M,Re
96339,0.047380,0.080088,0.072908,0.041155,0.012554,-0.020241,-0.018954,-0.015398,-0.011583,-0.004600,2.0,0.3,300000
13240,0.016694,0.024736,0.022058,0.013163,0.004284,-0.016694,-0.024736,-0.022058,-0.013163,-0.004284,-0.0,0.1,100000
49664,0.072448,0.109192,0.097923,0.057722,0.018512,-0.061304,-0.088714,-0.078543,-0.047595,-0.015752,10.0,0.1,500000
195756,0.132697,0.104186,0.031919,-0.015464,-0.010499,0.053716,-0.050304,-0.103421,-0.095133,-0.037371,5.0,0.1,200000
219800,0.124699,0.149966,0.111743,0.060361,0.017723,0.043334,0.050287,0.022558,0.007080,0.000402,4.0,0.3,200000
63090,0.098113,0.129986,0.102672,0.052489,0.015349,-0.058570,-0.117012,-0.119685,-0.080409,-0.027930,-4.0,0.2,500000
134839,0.161214,0.207909,0.170221,0.093411,0.028148,-0.021457,-0.088130,-0.098929,-0.067641,-0.024148,9.0,0.1,100000
235221,0.037522,0.059956,0.056694,0.035262,0.012574,-0.029343,-0.039015,-0.031538,-0.017402,-0.004564,-10.0,0.3,200000
183534,0.190848,0.272148,0.222440,0.105637,0.026151,-0.018009,-0.021588,-0.054515,-0.058237,-0.026198,5.0,0.3,500000
7646,0.066841,0.099001,0.088267,0.052665,0.017139,-0.066714,-0.098883,-0.088200,-0.052642,-0.017135,-8.0,0.1,500000


In [11]:
# displaying the targets table
y.head(10)

,CL,CD,Cm
96339,0.571,0.01533,-0.056
13240,0.000,0.01336,-0.000
49664,1.399,0.02784,-0.032
195756,-0.183,0.01525,0.359
219800,1.371,0.02474,-0.062
63090,-0.774,0.00994,0.109
134839,1.709,0.04955,0.028
235221,-1.109,0.02621,-0.022
183534,1.534,0.04382,0.034
7646,-1.028,0.02365,0.012


## Splitting the Dataset

In [12]:
# splitting dataset into training, validation & testing dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
# normalizing the training, validation & testing datasets
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_val = X_scaler.transform(X_val)
X_test = X_scaler.transform(X_test)

In [14]:
# displaying X_train, X_val & X_test after standardization
display(pd.DataFrame(X_train).head())
display(pd.DataFrame(X_val).head())
display(pd.DataFrame(X_test).head())

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.256021,-0.430293,-0.527753,-0.601280,-0.603619,-0.213933,-0.441463,-0.550288,-0.624228,-0.609182,-1.652291,0.001817,0.710313
1,0.435230,1.085944,1.353784,0.945204,-0.102321,0.830200,1.799003,2.159995,1.665016,0.669139,-0.991404,-1.223262,-1.410343
2,0.430090,0.587451,0.714598,0.896287,1.092395,-1.933811,-1.670076,-1.548224,-1.379254,-1.219591,0.495593,-1.223262,0.710313
3,-0.415138,-0.740797,-0.727770,-0.623304,-0.543888,1.076605,0.749459,0.798633,0.931932,1.028998,-0.826182,0.001817,0.710313
4,-0.218909,-1.014988,-1.505103,-1.936076,-2.156585,1.274748,0.446495,-0.024087,-0.444474,-0.614554,0.826037,0.001817,0.003428


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.435230,1.085944,1.353784,0.945204,-0.102321,0.830200,1.799003,2.159995,1.665016,0.669139,0.495593,1.226897,-0.703458
1,-0.752098,-0.592428,-0.435642,-0.308001,-0.205097,-0.135103,0.145686,0.336746,0.485779,0.595799,-0.330516,-1.223262,1.417198
2,0.595816,0.240209,0.003899,0.000096,-0.008399,1.834835,1.728250,1.553861,1.574516,1.557344,-0.991404,-1.223262,0.003428
3,-1.141392,-1.051673,-0.950854,-0.814374,-0.708529,0.220161,0.434995,0.570033,0.738204,0.869822,-0.330516,-1.223262,0.710313
4,-0.508005,-0.368678,-0.178369,0.039335,0.307969,-0.637101,-0.380273,-0.165585,0.068834,0.332781,0.165149,0.001817,1.417198


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.365737,-0.259663,-0.201216,-0.133761,-0.063813,-0.476259,-0.268781,-0.192159,-0.114692,-0.043020,0.660815,0.001817,1.417198
1,1.089976,0.847802,0.778043,0.861373,0.948130,0.509838,0.105783,0.013793,0.082511,0.135842,-1.321848,-1.223262,0.003428
2,-0.365737,-0.259663,-0.201216,-0.133761,-0.063813,-0.476259,-0.268781,-0.192159,-0.114692,-0.043020,-0.495738,0.001817,0.710313
3,0.431760,0.589212,0.717163,0.900372,1.101889,-1.931930,-1.668297,-1.545542,-1.374982,-1.210108,-0.660960,1.226897,0.003428
4,-1.664488,-1.645106,-1.495373,-1.336856,-1.224830,0.423341,0.587802,0.772453,0.972755,1.130601,1.486924,1.226897,-0.703458


In [15]:
# displaying y_train, y_val & y_test after standardization
display(pd.DataFrame(y_train).head())
display(pd.DataFrame(y_val).head())
display(pd.DataFrame(y_test).head())

,CL,CD,Cm
62853,-1.524,0.01376,0.112
230794,0.169,0.02784,-0.102
26221,0.429,0.02999,-0.007
149168,-0.291,0.01633,-0.012
129102,0.106,0.01142,0.226


,CL,CD,Cm
231139,1.376,0.03069,-0.108
73487,-0.030,0.01609,-0.033
219706,0.093,0.04335,-0.075
44591,-0.117,0.01377,-0.014
235820,0.276,0.01952,-0.032


,CL,CD,Cm
47453,0.619,0.02109,-0.002
154394,-0.419,0.01891,-0.026
47425,-0.298,0.02008,0.009
19893,-0.598,0.03323,0.010
145,1.071,0.02079,-0.005


In [16]:
# defining the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, min_lr=0.00001)

## Network Performance Due to Multiple Hidden Layers

### Case 1: Model [64, 3]

In [17]:
# RMSE and R² metric scores on test data for Model M_64
M_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64...")

    M_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_df = pd.DataFrame(M_64_metrics)

print(results_M_64_df.mean())



Run 1/20 for Model M_64...


I0000 00:00:1764131065.245543 2372590 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4770 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1764131066.915696 2372703 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Run 2/20 for Model M_64...

Run 3/20 for Model M_64...

Run 4/20 for Model M_64...

Run 5/20 for Model M_64...

Run 6/20 for Model M_64...

Run 7/20 for Model M_64...

Run 8/20 for Model M_64...

Run 9/20 for Model M_64...

Run 10/20 for Model M_64...

Run 11/20 for Model M_64...

Run 12/20 for Model M_64...

Run 13/20 for Model M_64...

Run 14/20 for Model M_64...

Run 15/20 for Model M_64...

Run 16/20 for Model M_64...

Run 17/20 for Model M_64...

Run 18/20 for Model M_64...

Run 19/20 for Model M_64...

Run 20/20 for Model M_64...
CL_rmse    0.005173
CD_rmse    0.005346
Cm_rmse    0.002402
CL_r2      0.999969
CD_r2      0.758661
Cm_r2      0.999329
dtype: float32


### Case 2: Model [64, 32, 3]

In [18]:
# RMSE and R² metric scores on test data for Model M_64_32
M_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32...")
    
    M_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_df = pd.DataFrame(M_64_32_metrics)

print(results_M_64_32_df.mean())



Run 1/20 for Model M_64_32...

Run 2/20 for Model M_64_32...

Run 3/20 for Model M_64_32...

Run 4/20 for Model M_64_32...

Run 5/20 for Model M_64_32...

Run 6/20 for Model M_64_32...

Run 7/20 for Model M_64_32...

Run 8/20 for Model M_64_32...

Run 9/20 for Model M_64_32...

Run 10/20 for Model M_64_32...

Run 11/20 for Model M_64_32...

Run 12/20 for Model M_64_32...

Run 13/20 for Model M_64_32...

Run 14/20 for Model M_64_32...

Run 15/20 for Model M_64_32...

Run 16/20 for Model M_64_32...

Run 17/20 for Model M_64_32...

Run 18/20 for Model M_64_32...

Run 19/20 for Model M_64_32...

Run 20/20 for Model M_64_32...
CL_rmse    0.003965
CD_rmse    0.005013
Cm_rmse    0.002007
CL_r2      0.999982
CD_r2      0.787819
Cm_r2      0.999533
dtype: float32


### Case 3: Model [64, 32, 16, 3]

In [19]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")
    
    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.003970
CD_rmse    0.005053
Cm_rmse    0.002073
CL_r2      0.999982
CD_r2      0.784293
Cm_r2      0.999499
dtype: float32


### Case 4: Model [64, 32, 16, 8, 3]

In [20]:
# RMSE and R² metric scores on test data for Model M_64_32_16_8
M_64_32_16_8_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16_8...")
    
    M_64_32_16_8 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16_8.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16_8.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16_8.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_8_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_8_df = pd.DataFrame(M_64_32_16_8_metrics)

print(results_M_64_32_16_8_df.mean())



Run 1/20 for Model M_64_32_16_8...

Run 2/20 for Model M_64_32_16_8...

Run 3/20 for Model M_64_32_16_8...

Run 4/20 for Model M_64_32_16_8...

Run 5/20 for Model M_64_32_16_8...

Run 6/20 for Model M_64_32_16_8...

Run 7/20 for Model M_64_32_16_8...

Run 8/20 for Model M_64_32_16_8...

Run 9/20 for Model M_64_32_16_8...

Run 10/20 for Model M_64_32_16_8...

Run 11/20 for Model M_64_32_16_8...

Run 12/20 for Model M_64_32_16_8...

Run 13/20 for Model M_64_32_16_8...

Run 14/20 for Model M_64_32_16_8...

Run 15/20 for Model M_64_32_16_8...

Run 16/20 for Model M_64_32_16_8...

Run 17/20 for Model M_64_32_16_8...

Run 18/20 for Model M_64_32_16_8...

Run 19/20 for Model M_64_32_16_8...

Run 20/20 for Model M_64_32_16_8...
CL_rmse    0.003964
CD_rmse    0.005033
Cm_rmse    0.002061
CL_r2      0.999981
CD_r2      0.785979
Cm_r2      0.999508
dtype: float32


In [21]:
M_64_stats = results_M_64_df.mean()
M_64_32_stats = results_M_64_32_df.mean()
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_64_32_16_8_stats = results_M_64_32_16_8_df.mean()

architectures = [
    '64, 3',
    '64, 32, 3',
    '64, 32, 16, 3',
    '64, 32, 16, 8, 3'
]

table_data = [
    (M_64_stats['CL_rmse'], M_64_stats['CD_rmse'], M_64_stats['Cm_rmse'],
     M_64_stats['CL_r2'], M_64_stats['CD_r2'], M_64_stats['Cm_r2']),
    
    (M_64_32_stats['CL_rmse'], M_64_32_stats['CD_rmse'], M_64_32_stats['Cm_rmse'],
     M_64_32_stats['CL_r2'], M_64_32_stats['CD_r2'], M_64_32_stats['Cm_r2']),
    
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_64_32_16_8_stats['CL_rmse'], M_64_32_16_8_stats['CD_rmse'], M_64_32_16_8_stats['Cm_rmse'],
     M_64_32_16_8_stats['CL_r2'], M_64_32_16_8_stats['CD_r2'], M_64_32_16_8_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 5), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))


## Network Performance Due to Increasing Neurons

### Case 1: Model [64, 32, 16, 3]

In [22]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")

    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.003971
CD_rmse    0.004966
Cm_rmse    0.002019
CL_r2      0.999982
CD_r2      0.791679
Cm_r2      0.999527
dtype: float32


### Case 2: Model [128, 64, 32, 3]

In [23]:
# RMSE and R² metric scores on test data for Model M_128_64_32
M_128_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_128_64_32...")

    M_128_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_128_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_128_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_128_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_128_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_128_64_32_df = pd.DataFrame(M_128_64_32_metrics)

print(results_M_128_64_32_df.mean())



Run 1/20 for Model M_128_64_32...

Run 2/20 for Model M_128_64_32...

Run 3/20 for Model M_128_64_32...

Run 4/20 for Model M_128_64_32...

Run 5/20 for Model M_128_64_32...

Run 6/20 for Model M_128_64_32...

Run 7/20 for Model M_128_64_32...

Run 8/20 for Model M_128_64_32...

Run 9/20 for Model M_128_64_32...

Run 10/20 for Model M_128_64_32...

Run 11/20 for Model M_128_64_32...

Run 12/20 for Model M_128_64_32...

Run 13/20 for Model M_128_64_32...

Run 14/20 for Model M_128_64_32...

Run 15/20 for Model M_128_64_32...

Run 16/20 for Model M_128_64_32...

Run 17/20 for Model M_128_64_32...

Run 18/20 for Model M_128_64_32...

Run 19/20 for Model M_128_64_32...

Run 20/20 for Model M_128_64_32...
CL_rmse    0.002889
CD_rmse    0.004487
Cm_rmse    0.001356
CL_r2      0.999990
CD_r2      0.829923
Cm_r2      0.999786
dtype: float32


### Case 3: Model [256, 128, 64, 3]

In [24]:
# RMSE and R² metric scores on test data for Model M_256_128_64
M_256_128_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_256_128_64...")

    M_256_128_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_256_128_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_256_128_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_256_128_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_256_128_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_256_128_64_df = pd.DataFrame(M_256_128_64_metrics)

print(results_M_256_128_64_df.mean())



Run 1/20 for Model M_256_128_64...

Run 2/20 for Model M_256_128_64...

Run 3/20 for Model M_256_128_64...

Run 4/20 for Model M_256_128_64...

Run 5/20 for Model M_256_128_64...

Run 6/20 for Model M_256_128_64...

Run 7/20 for Model M_256_128_64...

Run 8/20 for Model M_256_128_64...

Run 9/20 for Model M_256_128_64...

Run 10/20 for Model M_256_128_64...

Run 11/20 for Model M_256_128_64...

Run 12/20 for Model M_256_128_64...

Run 13/20 for Model M_256_128_64...

Run 14/20 for Model M_256_128_64...

Run 15/20 for Model M_256_128_64...

Run 16/20 for Model M_256_128_64...

Run 17/20 for Model M_256_128_64...

Run 18/20 for Model M_256_128_64...

Run 19/20 for Model M_256_128_64...

Run 20/20 for Model M_256_128_64...
CL_rmse    0.002704
CD_rmse    0.004186
Cm_rmse    0.001117
CL_r2      0.999991
CD_r2      0.851863
Cm_r2      0.999852
dtype: float32


### Case 4: Model [512, 256, 128, 3]

In [25]:
# RMSE and R² metric scores on test data for Model M_512_256_128
M_512_256_128_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_512_256_128...")

    M_512_256_128 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_512_256_128.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_512_256_128.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_512_256_128.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_512_256_128_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_512_256_128_df = pd.DataFrame(M_512_256_128_metrics)

print(results_M_512_256_128_df.mean())



Run 1/20 for Model M_512_256_128...

Run 2/20 for Model M_512_256_128...

Run 3/20 for Model M_512_256_128...

Run 4/20 for Model M_512_256_128...

Run 5/20 for Model M_512_256_128...

Run 6/20 for Model M_512_256_128...

Run 7/20 for Model M_512_256_128...

Run 8/20 for Model M_512_256_128...

Run 9/20 for Model M_512_256_128...

Run 10/20 for Model M_512_256_128...

Run 11/20 for Model M_512_256_128...

Run 12/20 for Model M_512_256_128...

Run 13/20 for Model M_512_256_128...

Run 14/20 for Model M_512_256_128...

Run 15/20 for Model M_512_256_128...

Run 16/20 for Model M_512_256_128...

Run 17/20 for Model M_512_256_128...

Run 18/20 for Model M_512_256_128...

Run 19/20 for Model M_512_256_128...

Run 20/20 for Model M_512_256_128...
CL_rmse    0.002453
CD_rmse    0.003878
Cm_rmse    0.000903
CL_r2      0.999993
CD_r2      0.872907
Cm_r2      0.999905
dtype: float32


### Case 5: Model [1024, 512, 256, 3]

In [26]:
# RMSE and R² metric scores on test data for Model M_1024_512_256
M_1024_512_256_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_1024_512_256...")

    M_1024_512_256 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_1024_512_256.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_1024_512_256.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_1024_512_256.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_1024_512_256_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_1024_512_256_df = pd.DataFrame(M_1024_512_256_metrics)

print(results_M_1024_512_256_df.mean())



Run 1/20 for Model M_1024_512_256...

Run 2/20 for Model M_1024_512_256...

Run 3/20 for Model M_1024_512_256...

Run 4/20 for Model M_1024_512_256...

Run 5/20 for Model M_1024_512_256...

Run 6/20 for Model M_1024_512_256...

Run 7/20 for Model M_1024_512_256...

Run 8/20 for Model M_1024_512_256...

Run 9/20 for Model M_1024_512_256...

Run 10/20 for Model M_1024_512_256...

Run 11/20 for Model M_1024_512_256...

Run 12/20 for Model M_1024_512_256...

Run 13/20 for Model M_1024_512_256...

Run 14/20 for Model M_1024_512_256...

Run 15/20 for Model M_1024_512_256...

Run 16/20 for Model M_1024_512_256...

Run 17/20 for Model M_1024_512_256...

Run 18/20 for Model M_1024_512_256...

Run 19/20 for Model M_1024_512_256...

Run 20/20 for Model M_1024_512_256...
CL_rmse    0.002256
CD_rmse    0.004002
Cm_rmse    0.000952
CL_r2      0.999994
CD_r2      0.863891
Cm_r2      0.999891
dtype: float32


In [27]:
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_128_64_32_stats = results_M_128_64_32_df.mean()
M_256_128_64_stats = results_M_256_128_64_df.mean()
M_512_256_128_stats = results_M_512_256_128_df.mean()
M_1024_512_256_stats = results_M_1024_512_256_df.mean()

architectures = [
    '64, 32, 16, 3',
    '128, 64, 32, 3',
    '256, 128, 64, 3',
    '512, 256, 128, 3',
    '1024, 512, 256, 3'
]

table_data = [
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_128_64_32_stats['CL_rmse'], M_128_64_32_stats['CD_rmse'], M_128_64_32_stats['Cm_rmse'],
     M_128_64_32_stats['CL_r2'], M_128_64_32_stats['CD_r2'], M_128_64_32_stats['Cm_r2']),
    
    (M_256_128_64_stats['CL_rmse'], M_256_128_64_stats['CD_rmse'], M_256_128_64_stats['Cm_rmse'],
     M_256_128_64_stats['CL_r2'], M_256_128_64_stats['CD_r2'], M_256_128_64_stats['Cm_r2']),
    
    (M_512_256_128_stats['CL_rmse'], M_512_256_128_stats['CD_rmse'], M_512_256_128_stats['Cm_rmse'],
     M_512_256_128_stats['CL_r2'], M_512_256_128_stats['CD_r2'], M_512_256_128_stats['Cm_r2']),
    
    (M_1024_512_256_stats['CL_rmse'], M_1024_512_256_stats['CD_rmse'], M_1024_512_256_stats['Cm_rmse'],
     M_1024_512_256_stats['CL_r2'], M_1024_512_256_stats['CD_r2'], M_1024_512_256_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 6), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))
